In [1]:
import json
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("error")
import matplotlib.pyplot as plt

In [2]:
# This cell prints the number of line segments defining cratons in the input
# file smac_cratons_coordinates.txt digitized from the paper by Nataf and 
# Richard, Physics of the Earth and Planetary Interiors, 1996.   
# In the input file, ">" separates the coordinates of the polygon representing
# cratons. We use the ">" separator in awk to find out number of line segments
# which are the number of cratons for the worldbuilder.
craton_file_name  = "smac_cratons_coordinates.txt"
process           = os.popen('''awk 'BEGIN {RS=">"} END{print NR}' ''' + craton_file_name)
number_of_cratons = process.read()
number_of_cratons = int(number_of_cratons)
process.close()

In [33]:
# This function defines the characteristics of the input cratons such as the 
# name, coordinates (used from the input file), and depth 
# to make sure they are defined appropriately for our global models.
def make_continental_json(name, coordinates):
    out = {
    "model": "continental plate",
    "name": name,
    "coordinates": coordinates,
    "min depth": -1e3,
    "max depth": 300e3,

    # temperature, composition, etc. would go here"
    # max depth chosen as the max continental plate thickness
    # hard coded the composition here
    "composition models": [{"model":"uniform", "compositions":[5]}]

     }
    return out

In [34]:
#--------------------------------------------------------------------
# This cell formats data so that the input coordinates are strings 
# in a list with each list item correspoding to 'long lat'.
# Awk initializes with 1 unlike python which has zero-index numbering. 
# The first record does not exist, therefore start with the 2 until 
# the number of faults + 1 for calling line segments in awk.
# After formatting the coordinates from the input file, we store each
# feature according to the make_continental_json() function to get
# polygons defining the cratons as needed in the world builder format.
#--------------------------------------------------------------------

craton_coordinates = []
all_cratons_json = []

for i in range (2, number_of_cratons + 1):
    process = os.popen('''awk 'BEGIN {FS=" "; OFS=","; RS=">"} {$1=$1}1 {if (NR==%s) print $0}' ''' \
                  % (str(i)) + craton_file_name)
    coordinates = process.read()
    process.close()
    # This is modification of the input strings in the coordinates to floats.
    coordinates_list   = (coordinates.rstrip('\n')).split(',')  
    list_of_floats     = [float(item) for item in coordinates_list]
    longitudes         = list_of_floats[0::2]
    longitudes_array   = np.array(longitudes)
    latitudes          = list_of_floats[1::2]
    coordinates_array  = [[longitudes[n],latitudes[n]] for n in range(len(longitudes))]
    
    all_cratons_json.append(make_continental_json('craton_' + str(i), coordinates_array))

In [35]:
# We add the fault features to the cratons because we want to include both plate boundaries
# and cratons. The parameter below defines which existing plate boundary model to use. 
wb_faults_file_name = '../input_data/world_builder_nuvel_boundaries.json'

with open(wb_faults_file_name) as f:
    gg = json.load(f)

faults = gg['features']
faults = [f for f in faults]

In [36]:
# This cell adds all the faults in 'features' that already contains the cratons
for fault in faults:
    all_cratons_json.append(fault)

In [41]:
all_features_json = {
    "version": "0.5",
    "interpolation": "continuous monotone spline",
    "coordinate system":{"model":"spherical", "depth method":"starting point"},
    "features": all_cratons_json
}

In [43]:
with open("../input_data/world_builder_smac_cratons_faults.json", "w") as f:
    json.dump(all_features_json, f)